In [15]:
###  THIS NOTEBOOK IS USED TO PROCESS THE GAME LEVEL DATA AND OUTPUT A YEARLY AGGREGATED DATAFRAME

# Importing libraries
import pandas as pd
import numpy as np
import os
import glob
import re
import datetime
import time
import tqdm
from tqdm import tqdm_notebook

# Setting the working directory


In [16]:
## Setting the working directory
# 'C:\Users\Justin\Desktop\Project\BB_CLEAN\BK_FB_Playground\TEMP\'

# Setting the working directory
os.chdir('C:\\Users\\Justin\\Desktop\\Project\\BB_CLEAN\\BK_FB_Playground\\TEMP\\')

file = 'FB_SCRAPE_v1.csv'

# Reading in the data
df = pd.read_csv(file)

# Checking the data
df.head()

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56387 entries, 0 to 56386
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   TeamName          56387 non-null  object 
 1   PopularName       56387 non-null  object 
 2   LeagueId          56387 non-null  int64  
 3   LeagueName        53698 non-null  object 
 4   LeagueAbbr        44185 non-null  object 
 5   SeeasonYear       56387 non-null  int64  
 6   Date              56387 non-null  object 
 7   Opponent          56203 non-null  object 
 8   Location          56387 non-null  object 
 9   Result            55854 non-null  object 
 10  PrimaryTeamScore  55849 non-null  float64
 11  PrimaryTeamNote   1205 non-null   object 
 12  OpponentScore     55849 non-null  float64
 13  OpponentNote      837 non-null    object 
 14  IsLeagueGame      56387 non-null  bool   
 15  ContestType       56387 non-null  int64  
 16  SeasonType        56387 non-null  int64 

In [17]:
### EXAMINE THE DATA

# Checking the data

# How many team are represented in the data?
df['TeamName'].nunique()


# How many years are represented in the data?
df['SeeasonYear'].nunique()

# How many games have results vs no results?
df['Result'].value_counts()

# how many nan values are there in the data?
df.isna().sum()




TeamName                0
PopularName             0
LeagueId                0
LeagueName           2689
LeagueAbbr          12202
SeeasonYear             0
Date                    0
Opponent              184
Location                0
Result                533
PrimaryTeamScore      538
PrimaryTeamNote     55182
OpponentScore         538
OpponentNote        55550
IsLeagueGame            0
ContestType             0
SeasonType              0
TournamentName      51115
GameStatusType          0
GameNotes           50963
dtype: int64

In [18]:
### How many games are there per year?
df.groupby('SeeasonYear')['TeamName'].nunique()


# How many playoff vs regular season games are there per year?
df.groupby(['SeeasonYear','SeasonType'])['TeamName'].nunique()



SeeasonYear  SeasonType
2012         1             468
             3             268
2013         1             580
             3             253
2014         1             573
             3             252
2015         1             561
             3             252
2016         1             552
             3             251
2017         1             539
             3             251
2018         1             534
             3             253
2019         1             519
             3             251
2020         1             497
             3             484
2021         1             491
             3             250
2022         1             481
             3             251
Name: TeamName, dtype: int64

In [19]:
df.sample(5)

## location values
# df['Location'].value_counts()

# df.info()

,TeamName,PopularName,LeagueId,LeagueName,LeagueAbbr,SeeasonYear,Date,Opponent,Location,Result,PrimaryTeamScore,PrimaryTeamNote,OpponentScore,OpponentNote,IsLeagueGame,ContestType,SeasonType,TournamentName,GameStatusType,GameNotes
41718,Hudson,Hudson,6939,Lenawee County Athletic Assn,Lenawee County,2019,2019-08-30T19:30:00,Ithaca,A,L,24.0,NaN,38.0,NaN,False,1,1,NaN,NOR,NaN
26805,Melvindale Academy for Business & Tech,Melvindale Academy for Business & Tech,9502,Michigan Metro Athletic Conference-Black,MMAC-Black,2017,2017-09-29T19:00:00,Ecorse,A,L,0.0,NaN,55.0,NaN,True,1,1,NaN,NOR,NaN
595,Auburn Hills Avondale,Auburn Hills Avondale,7024,Oakland Activities Association-Blue,OAA Blue,2012,2012-09-21T19:00:00,Berkley,H,W,23.0,NaN,21.0,NaN,True,1,1,NaN,NOR,NaN
38891,North Farmington,North Farmington,7024,Oakland Activities Association-Blue,OAA Blue,2019,2019-08-29T19:00:00,Waterford Kettering,H,W,35.0,NaN,30.0,NaN,False,1,1,NaN,NOR,NaN
16965,Mattawan,Mattawan,7016,Southwestern MI Ath Conf-East,Big 16 East,2015,2015-10-16T19:00:00,Stevensville Lakeshore,H,L,14.0,NaN,35.0,NaN,False,1,1,NaN,NOR,NaN


In [20]:
## Calculate the point differential for each game
## aboslute value of the difference between the team's score and the opponent's score
df['PointDiff'] = df['PrimaryTeamScore'] - df['OpponentScore']






In [21]:
## IsLeagueGame datatype
df['IsLeagueGame'] = df['IsLeagueGame'].astype('bool')

In [22]:
## Create the slices

# regular season
df_reg = df[df['SeasonType'] == 1]

# playoff
df_playoff = df[df['SeasonType'] == 3]


# home - only regular season
# Call from the regular season slice
df_home = df_reg[df_reg['Location'] == 'H']
df_away = df_reg[df_reg['Location'] == 'A']

# only winning / losing games
df_win = df[df['Result'] == 'W']
df_loss = df[df['Result'] == 'L']
             
# only close games - games with a point of +/- 7
df_close = df[(df['PointDiff'] >= -7) & (df['PointDiff'] <= 7)]



# only blowout wins - games with a point differential of + 21
df_blowout_w = df[df['PointDiff'] >= 21]

# only blowout losses - games with a point differential of - 21
df_blowout_l = df[df['PointDiff'] <= -21]





# only league games
df_league = df[df['IsLeagueGame'] == True]
# non league games
df_nonleague = df[df['IsLeagueGame'] == False]








In [23]:
## Check the length of each slice
print('Regular Season: ', len(df_reg))
print('Playoff: ', len(df_playoff))
print('Home: ', len(df_home))
print('Away: ', len(df_away))
print('Win: ', len(df_win))
print('Loss: ', len(df_loss))
print('Close: ', len(df_close))
print('Blowout_W: ', len(df_blowout_w))
print('Blowout_L: ', len(df_blowout_l))
print('League: ', len(df_league))
print('Non-League: ', len(df_nonleague))



Regular Season:  50398
Playoff:  5989
Home:  25411
Away:  24987
Win:  27356
Loss:  27286
Close:  11275
Blowout_W:  15416
Blowout_L:  15357
League:  31277
Non-League:  25110


In [24]:
## Create a summary dataframe with year by year stats for each team

## Overall statistics I want to capture and how I want them sliced.

# Make slices out of each season on 'SeeasonYear' and calculate summary statistics
# want to make slices out of regular season vs playoff games on 'SeasonType'
# want to make slices out of home vs away games on 'Location' - only for regular season games
# I would like the same set of stats for each of the above slices and overall

# summary of Reults (W/L/T/WF/LF)
# points scored, allowed and point differential - overall and at home vs away, regular season vs postseason, in wins vs losses, in close games vs blowouts, in league games vs non-league games

# winning percentage
# number of games played
# number of games won
# number of games lost
# number of games tied
# forfeit games won or lost
# overtime games (won, lost)

# Close games won and lost (score diff of 7 points or less)
# blowouts won and lost (score diff of 21 points or more)







In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56387 entries, 0 to 56386
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   TeamName          56387 non-null  object 
 1   PopularName       56387 non-null  object 
 2   LeagueId          56387 non-null  int64  
 3   LeagueName        53698 non-null  object 
 4   LeagueAbbr        44185 non-null  object 
 5   SeeasonYear       56387 non-null  int64  
 6   Date              56387 non-null  object 
 7   Opponent          56203 non-null  object 
 8   Location          56387 non-null  object 
 9   Result            55854 non-null  object 
 10  PrimaryTeamScore  55849 non-null  float64
 11  PrimaryTeamNote   1205 non-null   object 
 12  OpponentScore     55849 non-null  float64
 13  OpponentNote      837 non-null    object 
 14  IsLeagueGame      56387 non-null  bool   
 15  ContestType       56387 non-null  int64  
 16  SeasonType        56387 non-null  int64 

In [26]:
##########  CREATE A FUNCTION TO CALCULATE THE STATS FOR EACH SLICE ##########

def create_summary_stats(df, df_name):
    # group by TeamName and SeeasonYear
    df_grouped = df.groupby(['TeamName','SeeasonYear','LeagueAbbr'])

    # create a dataframe to store the summary stats
    df_summary = pd.DataFrame()

    # create a list of the stats I want to calculate
    stats = ['Win','Loss','Tie','WinForfeit','LossForfeit','OvertimeWin','OvertimeLoss',
            'PointsScored','PointsAllowed','PointDiff', 
            'AVGPointsScored', 'AVGPointsAllowed', 'AVGPointDiff']
    
    ## Deifne the stats to calculate
    # number of games played
    df_summary['GamesPlayed'] = df_grouped['TeamName'].count()
    # number of games won
    df_summary['Win'] = df_grouped['Result'].apply(lambda x: (x == 'W').sum())
    # number of games lost
    df_summary['Loss'] = df_grouped['Result'].apply(lambda x: (x == 'L').sum())
    # number of games tied
    df_summary['Tie'] = df_grouped['Result'].apply(lambda x: (x == 'T').sum())
    # Win Percentage
    df_summary['Win_Pct'] = df_summary['Win'] / df_summary['GamesPlayed']
    # forfeit games won
    df_summary['WinForfeit'] = df_grouped['Result'].apply(lambda x: (x == 'WF').sum())
    # forfeit games lost
    df_summary['LossForfeit'] = df_grouped['Result'].apply(lambda x: (x == 'LF').sum())
    # overtime games won
    df_summary['OvertimeWin'] = df_grouped['Result'].apply(lambda x: (x == 'OTW').sum())
    # overtime games lost
    df_summary['OvertimeLoss'] = df_grouped['Result'].apply(lambda x: (x == 'OTL').sum())

    # points scored
    df_summary['PointsScored'] = df_grouped['PrimaryTeamScore'].sum()    
    df_summary['AVGPointsScored'] = df_grouped['PrimaryTeamScore'].mean()

    # points allowed
    df_summary['PointsAllowed'] = df_grouped['OpponentScore'].sum()    
    df_summary['AVGPointsAllowed'] = df_grouped['OpponentScore'].mean()

    # point differential
    df_summary['PointDiff'] = df_grouped['PointDiff'].sum()
    df_summary['AVGPointDiff'] = df_grouped['PointDiff'].mean()

    # reset the index
    df_summary.reset_index(inplace=True)

    # rename the columns
    df_summary.rename(columns={'TeamName':'Team','SeeasonYear':'Year'}, inplace=True)

    # add a column for the season type
    df_summary['SeasonType'] = df_name

    # return the dataframe
    return df_summary



###################  RUNS AS A STANDALONE SCRIPT  #####################

In [27]:
############### RUN ON OVERALL AND THE LIST OF SLICES OF THE DATA #################

# create a list of the slices I want to make
slices = [df, df_reg, df_playoff, df_home, df_away, df_win, df_loss, df_close, df_blowout_w, df_blowout_l, df_league, df_nonleague]

# create a list of the names of the slices
slice_names = ['Overall','Regular Season','Playoff','Home','Away','Win','Loss','Close','Blowout_Wins','Blowout_Loses', 'League','Non-League']

# create a list to store the dataframes
df_list = []

# loop through the slices and create the summary stats
for i in range(len(slices)):
    df_list.append(create_summary_stats(slices[i], slice_names[i]))

# concatenate the dataframes
df_summary = pd.concat(df_list)

# reset the index
df_summary.reset_index(drop=True, inplace=True)

# save the dataframe to a csv file
df_summary.to_csv('../TEMP/summary_stats_v11.csv', index=False)






In [28]:
## Look at df_summary

df_summary.info()

df_summary.head()

df_summary.tail()

df_summary.describe()

df_summary['Year'].value_counts()
# 
df_summary['Team'].value_counts()

df_summary['SeasonType'].value_counts()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50820 entries, 0 to 50819
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Team              50820 non-null  object 
 1   Year              50820 non-null  int64  
 2   LeagueAbbr        50820 non-null  object 
 3   GamesPlayed       50820 non-null  int64  
 4   Win               50820 non-null  int64  
 5   Loss              50820 non-null  int64  
 6   Tie               50820 non-null  int64  
 7   Win_Pct           50820 non-null  float64
 8   WinForfeit        50820 non-null  int64  
 9   LossForfeit       50820 non-null  int64  
 10  OvertimeWin       50820 non-null  int64  
 11  OvertimeLoss      50820 non-null  int64  
 12  PointsScored      50820 non-null  float64
 13  AVGPointsScored   50805 non-null  float64
 14  PointsAllowed     50820 non-null  float64
 15  AVGPointsAllowed  50805 non-null  float64
 16  PointDiff         50820 non-null  float6

Overall           4754
Regular Season    4712
Non-League        4681
Loss              4658
Away              4643
Home              4619
League            4377
Win               4346
Close             3949
Blowout_Loses     3905
Blowout_Wins      3710
Playoff           2466
Name: SeasonType, dtype: int64

In [29]:
## Save the dataframe
df_summary.to_csv('NEW_TEST.csv', index=False)